In [14]:
import sqlite3

conn = sqlite3.connect("badge_login.db")
cursor = conn.cursor()


In [15]:
import sqlite3
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker()
conn = sqlite3.connect("badge_login.db")
cursor = conn.cursor()

# ---------------------------
# 1️⃣ Insert dummy departments
# ---------------------------
departments = [
    ("D01", "Accounting"),
    ("D02", "IT"),
    ("D03", "HR"),
    ("D04", "Construction")
]

for dept_id, dept_name in departments:
    cursor.execute("""
    INSERT OR IGNORE INTO dim_department (department_id, department_name)
    VALUES (?, ?)
    """, (dept_id, dept_name))

# ---------------------------
# 2️⃣ Insert dummy employees
# ---------------------------
# We'll fetch department keys first
cursor.execute("SELECT department_key, department_id FROM dim_department")
dept_map = {dept_id: dept_key for dept_key, dept_id in cursor.fetchall()}

employees = []
employment_types = ["Full-Time", "Part-Time"]

for i in range(1, 6):  # 5 employees
    dept_id = random.choice(list(dept_map.keys()))
    employees.append((
        f"E{i:03}",                       # employee_id
        fake.name(),                      # full_name
        dept_map[dept_id],                # department_key
        random.choice(["Analyst", "Manager", "Technician", "Worker"]),  # job_title
        random.choice(employment_types),  # employment_type
        round(random.uniform(15, 50), 2), # hourly_rate
        fake.date_between(start_date='-5y', end_date='today').isoformat(), # hire_date
        1                                 # is_active
    ))

cursor.executemany("""
INSERT OR IGNORE INTO dim_employee 
(employee_id, full_name, department_key, job_title, employment_type, hourly_rate, hire_date, is_active)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""", employees)

# ---------------------------
# 3️⃣ Insert dummy timecards
# ---------------------------
cursor.execute("SELECT employee_key, hourly_rate FROM dim_employee")
employee_data = cursor.fetchall()

for emp_key, rate in employee_data:
    # Generate 2 random shifts per employee
    for _ in range(2):
        work_date = fake.date_between(start_date='-30d', end_date='today').isoformat()
        # Random shift between 4 and 8 hours
        hours = random.choice([4, 6, 8])
        time_in_hour = random.randint(6, 14)  # shift start between 6am-2pm
        time_in = f"{time_in_hour:02}:00"
        time_out_hour = time_in_hour + hours
        time_out = f"{time_out_hour:02}:00"
        gross_pay = round(hours * rate, 2)
        
        cursor.execute("""
        INSERT INTO fact_timecards
        (employee_key, work_date, time_in, time_out, hours_worked, gross_pay)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (emp_key, work_date, time_in, time_out, hours, gross_pay))

# Commit changes and close
conn.commit()


In [18]:
cursor.execute("SELECT * FROM dim_employee")
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'E001', 'Carly Day', 3, 'Manager', 'Part-Time', 45.68, '2025-11-21', 1)
(2, 'E002', 'Alexander Craig', 4, 'Technician', 'Full-Time', 23.39, '2023-03-20', 1)
(3, 'E003', 'James Lloyd', 4, 'Manager', 'Part-Time', 29.67, '2024-01-26', 1)
(4, 'E004', 'Chad King', 2, 'Analyst', 'Part-Time', 31.21, '2023-08-22', 1)
(5, 'E005', 'Julia Hernandez', 2, 'Technician', 'Part-Time', 24.08, '2025-12-22', 1)


In [ ]:
conn.commit()
conn.close()